In [23]:
import pandas as pd
import numpy as np
from scipy import stats

import seaborn as sns
import pylab as plt

import sys
sys.path.insert(1,'../scripts')
import plots

In [3]:
ratesPC = pd.read_csv('/scratch/c.c21013066/data/ppmi/analyses/cluster/PC_clinicalbioleddmedasym_digital.csv',index_col=[0])

In [16]:
# correlate progression with PCs
#first_PC = PC_df.filter(regex='^PC').columns[:10]#[f'{p}_PC_{i}' for i in range(10) for p in predictors]
#ratesPC = pd.merge(rates,nona.set_index('participant').filter(regex='PC_'),right_index=True,left_index=True,how='outer')
#ratesPC = pd.merge(rates,PC_df.set_index('participant')[first_PC],right_index=True,left_index=True,how='outer')
items = ratesPC.filter(regex='_PC$').columns
labels = ['autonom','cognitive','daily','medication','motor','psychiatric']
filtered_list = [item for item in items if any(term in item for term in labels)]
filtered_list = [item for item in filtered_list if 'slope' in item]
corr = pd.DataFrame(index=pd.MultiIndex.from_product([filtered_list,ratesPC.filter(regex='^PC').columns[:10]],names=['clinical','digital'])
                    ,columns=['pearson r','p-value','N'])
for pc in ratesPC.filter(regex='^PC').columns[:10]:
    for c in filtered_list:
        df = ratesPC.dropna(subset=[pc,c])
        r,p = stats.pearsonr(df[c],df[pc])
        corr.loc[(c,pc),'pearson r'] = r
        corr.loc[(c,pc),'p-value'] = p
        corr.loc[(c,pc),'N'] = df.shape[0]
        
thr = 0.05/((corr[['pearson r']].unstack().astype(float).shape[1]/2))

corr['sign'] = corr['p-value']< thr
corr['sign'] = corr['sign'].replace([True,False],['*',''])

In [18]:
corr.to_csv('/scratch/c.c21013066/data/ppmi/analyses/progression/consensuscohort/correlation_digital_clinical.csv')